# The Impact of Litigations on GHG Emissions
* "litigation" from [Graham Institute](https://climate-laws.org/legislation_and_policies), takes the value of 1 when the first case related to environmental laws was solved in a given year, independently of the result
* "rule of law" from World Bank
* Sample excludes the US, and includes all countries for which there has been at least a case with a resolution: 23 countries, mostly HI and MI countries

In [1]:
options(conflicts.policy = list(warn = FALSE))
library("dplyr")
library("estimatr")
library("lmtest")
library("plm")
library("readr")
library("rstatix")

Loading required package: zoo



In [2]:
my_data <- read_csv("litigation.csv", show_col_types = 'FALSE')

In [3]:
# egen id = group(country)
my_data <- my_data %>% group_by(country) %>% mutate(id = cur_group_id())

In [4]:
# xtset id year
plm::is.pbalanced(my_data$id, my_data$year)

[1] TRUE

In [5]:
my_data$lnghg <- log(my_data$ghg)
my_data$lngdp <- log(my_data$gdp)
my_data$lngdp2 <- log(my_data$gdp)^2
my_data$lnpopulation <- log(my_data$population)
my_data$lnlaw <- log(my_data$ruleoflaw)

In [6]:
# sum lnghg lngdp lngdp2 lnpop lnlaw litigation
d <- data.frame(my_data$lnghg, my_data$lngdp, my_data$lngdp2, my_data$lnpopulation, my_data$lnlaw)
summary(d)

 my_data.lnghg   my_data.lngdp    my_data.lngdp2   my_data.lnpopulation
 Min.   :14.41   Min.   : 7.388   Min.   : 54.58   Min.   :12.85       
 1st Qu.:17.64   1st Qu.: 9.420   1st Qu.: 88.74   1st Qu.:15.45       
 Median :18.27   Median :10.323   Median :106.56   Median :16.39       
 Mean   :18.49   Mean   : 9.986   Mean   :100.67   Mean   :16.52       
 3rd Qu.:19.83   3rd Qu.:10.677   3rd Qu.:114.00   3rd Qu.:17.65       
 Max.   :21.94   Max.   :11.701   Max.   :136.90   Max.   :21.03       
                 NA's   :23       NA's   :23                           
 my_data.lnlaw  
 Min.   :2.664  
 1st Qu.:3.990  
 Median :4.367  
 Mean   :4.184  
 3rd Qu.:4.553  
 Max.   :4.605  
 NA's   :288    

### Table 3

In [7]:
# reg lnghg lngdp lngdp2 lnpopulation lnlaw litigation , robust
mod2 <-estimatr::lm_robust(lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + litigation, data = my_data, se_type = "stata")
summary(mod2)


Call:
estimatr::lm_robust(formula = lnghg ~ lngdp + lngdp2 + lnpopulation + 
    lnlaw + litigation, data = my_data, se_type = "stata")

Standard error type:  HC1 

Coefficients:
              Estimate Std. Error t value  Pr(>|t|)  CI Lower  CI Upper  DF
(Intercept)  -14.22820    1.44345  -9.857 2.027e-21 -17.06273 -11.39367 633
lngdp          2.57894    0.30684   8.405 2.832e-16   1.97639   3.18150 633
lngdp2        -0.09856    0.01627  -6.056 2.387e-09  -0.13052  -0.06661 633
lnpopulation   0.99947    0.01131  88.391 0.000e+00   0.97727   1.02168 633
lnlaw          0.07553    0.06750   1.119 2.636e-01  -0.05702   0.20807 633
litigation    -0.04786    0.03651  -1.311 1.903e-01  -0.11955   0.02382 633

Multiple R-squared:  0.9221 ,	Adjusted R-squared:  0.9215 
F-statistic:  1930 on 5 and 633 DF,  p-value: < 2.2e-16

In [8]:
# xtreg lnghg lngdp lngdp2 lnpopulation lnlaw litigation i.year, fe robust

modfe <- plm::plm(lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + litigation,
                  index = c("id", "year"), model="within", effect='twoways', data = my_data)
summary(modfe, vcov=plm::vcovHC(modfe))

Twoways effects Within Model

Note: Coefficient variance-covariance matrix supplied: plm::vcovHC(modfe)

Call:
plm::plm(formula = lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + 
    litigation, data = my_data, effect = "twoways", model = "within", 
    index = c("id", "year"))

Unbalanced Panel: n = 32, T = 19-20, N = 639

Residuals:
      Min.    1st Qu.     Median    3rd Qu.       Max. 
-0.2205138 -0.0372824 -0.0013006  0.0396659  0.1973966 

Coefficients:
              Estimate Std. Error t-value  Pr(>|t|)    
lngdp         2.457919   0.530010  4.6375 4.357e-06 ***
lngdp2       -0.096579   0.028671 -3.3685 0.0008057 ***
lnpopulation  1.429588   0.172184  8.3027 7.123e-16 ***
lnlaw        -0.069259   0.034396 -2.0135 0.0445173 *  
litigation   -0.034449   0.015895 -2.1673 0.0306194 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    9.9414
Residual Sum of Squares: 2.8429
R-Squared:      0.71404
Adj. R-Squared: 0.68706
F-statistic: 55.0814

In [9]:
HI <- my_data$country %in% c('BEL', 'AUS', 'CAN', 'CHE', 'DEU', 'ESP', 'EST', 'FIN', 
                             'FRA', 'GBR', 'GRC', 'HRV', 'ITA', 'LTU', 'LUX', 'NLD', 
                             'NOR', 'NZL', 'POL', 'PRT', 'SVK', 'SWE', 'ROU')
my_data$HITRUE_litigation <- HI * my_data$litigation
my_data$HIFALSE_litigation <- (1-HI) * my_data$litigation

In [10]:
# reg lnghg lngdp lngdp2 lnpopulation lnlaw HI#c.litigation, robust

mod2 <-estimatr::lm_robust(lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + HITRUE_litigation + HIFALSE_litigation, data = my_data, se_type = "stata")
summary(mod2)


Call:
estimatr::lm_robust(formula = lnghg ~ lngdp + lngdp2 + lnpopulation + 
    lnlaw + HITRUE_litigation + HIFALSE_litigation, data = my_data, 
    se_type = "stata")

Standard error type:  HC1 

Coefficients:
                    Estimate Std. Error t value  Pr(>|t|)  CI Lower   CI Upper
(Intercept)        -13.99222    1.43584  -9.745 5.337e-21 -16.81181 -11.172628
lngdp                2.51508    0.30694   8.194 1.404e-15   1.91235   3.117820
lngdp2              -0.09540    0.01625  -5.870 7.057e-09  -0.12732  -0.063484
lnpopulation         0.99783    0.01136  87.809 0.000e+00   0.97551   1.020140
lnlaw                0.10214    0.06986   1.462 1.442e-01  -0.03504   0.239321
HITRUE_litigation   -0.07211    0.03908  -1.845 6.551e-02  -0.14886   0.004642
HIFALSE_litigation   0.08685    0.07225   1.202 2.298e-01  -0.05503   0.228719
                    DF
(Intercept)        632
lngdp              632
lngdp2             632
lnpopulation       632
lnlaw              632
HITRUE_litigation

In [11]:
# xtreg lnghg lngdp lngdp2 lnpopulation lnlaw HI#c.litigation i.year, fe robust

modfe <- plm::plm(lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + HITRUE_litigation + HIFALSE_litigation,
                  index = c("id", "year"), model="within", effect='twoways', data = my_data)
summary(modfe, vcov=plm::vcovHC(modfe))

Twoways effects Within Model

Note: Coefficient variance-covariance matrix supplied: plm::vcovHC(modfe)

Call:
plm::plm(formula = lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + 
    HITRUE_litigation + HIFALSE_litigation, data = my_data, effect = "twoways", 
    model = "within", index = c("id", "year"))

Unbalanced Panel: n = 32, T = 19-20, N = 639

Residuals:
       Min.     1st Qu.      Median     3rd Qu.        Max. 
-0.22312652 -0.03797838 -0.00071444  0.03988400  0.20408018 

Coefficients:
                    Estimate Std. Error t-value  Pr(>|t|)    
lngdp               2.326192   0.502026  4.6336 4.438e-06 ***
lngdp2             -0.090108   0.027112 -3.3236 0.0009445 ***
lnpopulation        1.430751   0.175797  8.1387 2.433e-15 ***
lnlaw              -0.069230   0.035009 -1.9775 0.0484574 *  
HITRUE_litigation  -0.040771   0.018069 -2.2564 0.0244154 *  
HIFALSE_litigation -0.010867   0.035185 -0.3088 0.7575470    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’